In [124]:
import numpy as np
import sympy as sp
from sympy import Matrix
from joblib import Parallel, delayed
import multiprocessing

In [141]:
import math

def generate_random_key_matrix(n, max_attempts=5000):
    for _ in range(max_attempts):
        key = np.random.randint(0, 26, (n, n))
        det = int(np.linalg.det(key))  # Convert determinant to integer
        print("Det:", det)
        if np.linalg.matrix_rank(key) == n and math.gcd(det, 26) == 1:
            return key
    raise ValueError("Maximum number of attempts reached. Unable to generate a suitable key matrix.")

In [142]:
generate_random_key_matrix(10)

Det: 4214943937477
Det: -6651546771202
Det: -1916405840511


array([[ 5, 17, 23, 12,  4, 18,  9, 11, 23, 10],
       [11,  9, 25,  3, 16, 17,  1, 25, 23, 25],
       [21, 22, 23,  8, 19, 12,  0, 13, 11,  5],
       [22, 12, 24, 14, 15,  6,  1, 11, 23, 24],
       [ 2,  5,  4, 18, 25, 25, 24, 13,  6, 12],
       [17, 18,  2, 24, 25,  5, 14, 11, 20, 12],
       [ 6, 10, 21, 15,  0, 13, 12, 18,  9,  8],
       [16, 11,  1,  7, 22,  9, 12, 23,  6, 17],
       [19,  6, 14, 17, 19, 11, 12, 24,  3, 22],
       [25, 10,  8, 25,  0,  5,  9,  3, 23,  0]])

In [127]:
# Function to encrypt a message using Hill Cipher
def hill_cipher_encrypt(plain_text, key):
    # Convert plain text to numbers
    plain_text = [ord(char) - ord('a') for char in plain_text.lower()]
    n = len(key)
    encrypted_text = ""

    # Pad plain text if its length is not a multiple of key size
    if len(plain_text) % n != 0:
        plain_text.extend([0] * (n - len(plain_text) % n))

    # Encrypt the text
    for i in range(0, len(plain_text), n):
        block = plain_text[i:i+n]
        encrypted_block = np.dot(key, block) % 26
        encrypted_text += ''.join([chr(char + ord('a')) for char in encrypted_block])

    return encrypted_text

In [128]:
# Function to decrypt a message using Hill Cipher
def hill_cipher_decrypt(encrypted_text, key):
    # Convert encrypted text to numbers
    encrypted_text = [ord(char) - ord('a') for char in encrypted_text.lower()]
    key_inv = Matrix(key).inv_mod(26)
    decrypted_text = ""

    # Decrypt the text
    for i in range(0, len(encrypted_text), len(key)):
        block = encrypted_text[i:i+len(key)]
        decrypted_block = (key_inv * Matrix(block)) % 26
        decrypted_text += ''.join([chr(int(char) + ord('a')) for char in decrypted_block])

    return decrypted_text

In [129]:
# Parallel version of Hill Cipher encryption
def parallel_hill_cipher_encrypt(plain_text, key, num_cores):
    split_text = [plain_text[i::num_cores] for i in range(num_cores)]
    encrypted_text_parts = Parallel(n_jobs=num_cores)(delayed(hill_cipher_encrypt)(text_part, key) for text_part in split_text)
    return ''.join(encrypted_text_parts)

In [130]:
# Parallel version of Hill Cipher decryption
def parallel_hill_cipher_decrypt(encrypted_text, key, num_cores):
    split_text = [encrypted_text[i::num_cores] for i in range(num_cores)]
    decrypted_text_parts = Parallel(n_jobs=num_cores)(delayed(hill_cipher_decrypt)(text_part, key) for text_part in split_text)
    return ''.join(decrypted_text_parts)

In [133]:
# Example usage
key = generate_random_key_matrix(2)
print("Generated Key Matrix:\n", key)
plain_text = "hello"
encrypted_text = parallel_hill_cipher_encrypt(plain_text, key, num_cores=4)
print("Encrypted Text:", encrypted_text)
decrypted_text = parallel_hill_cipher_decrypt(encrypted_text, key, num_cores=4)
print("Decrypted Text:", decrypted_text)

Det: -41
Generated Key Matrix:
 [[ 8 23]
 [ 7 15]]
Encrypted Text: ozgckzkz
Decrypted Text: kwwhsuty


In [134]:
import time
import matplotlib.pyplot as plt

In [144]:
def measure_serial_encryption_time(plain_text, key):
    start_time = time.time()
    encrypted_text = hill_cipher_encrypt(plain_text, key)
    end_time = time.time()
    return end_time - start_time

In [136]:
def measure_parallel_encryption_time(plain_text, key, num_cores):
    start_time = time.time()
    encrypted_text = parallel_hill_cipher_encrypt(plain_text, key, num_cores)
    end_time = time.time()
    return end_time - start_time

In [137]:
# Function to plot the graph for N vs Speed Up
def plot_speed_up_graph(N_values, serial_times, parallel_times):
    speed_up_data = [serial_times[i] / parallel_times[i] for i in range(len(N_values))]
    plt.figure(figsize=(8, 6))
    plt.plot(N_values, speed_up_data, marker='o', linestyle='-')
    plt.title('N vs Speed Up')
    plt.xlabel('N')
    plt.ylabel('Speed Up')
    plt.grid(True)
    plt.show()

In [138]:
# Function to plot the graph for N vs Parallel Efficiency
def plot_parallel_efficiency_graph(N_values, parallel_times):
    num_cores = 4  # Number of processing elements
    parallel_efficiency_data = [serial_times[i] / (parallel_times[i] * num_cores) for i in range(len(N_values))]
    plt.figure(figsize=(8, 6))
    plt.plot(N_values, parallel_efficiency_data, marker='o', linestyle='-')
    plt.title('N vs Parallel Efficiency')
    plt.xlabel('N')
    plt.ylabel('Parallel Efficiency')
    plt.grid(True)
    plt.show()

In [145]:
def generate_data():
    N_values = [5, 10, 15, 20, 25]  # Different values of N
    serial_times = []
    parallel_times = []

    for N in N_values:
        key = generate_random_key_matrix(N)
        plain_text = "hello" * (N // 5)  # Adjusting plain text length for each value of N
        
        # Measure serial encryption time
        serial_time = measure_serial_encryption_time(plain_text, key)
        
        # Measure parallel encryption time
        parallel_time = measure_parallel_encryption_time(plain_text, key, num_cores=4)
        
        serial_times.append(serial_time)
        parallel_times.append(parallel_time)

    return N_values, serial_times, parallel_times

In [146]:
# Generate data
N_values, serial_times, parallel_times = generate_data()

# Plot graphs
# plot_speed_up_graph(N_values, serial_times, parallel_times)
# plot_parallel_efficiency_graph(N_values, parallel_times)

Det: -737134
Det: -428039
Det: 1618177154335
Det: 17361604258982946816
Det: -6428532366909699072
Det: 17997530965937031168
Det: -103649343216418783232
Det: -3099560173946137088
Det: -244519429637251629056
Det: -54332087647669993472
Det: -11489699036414652416
Det: 5308527542949293056
Det: -7987476518705563648
Det: -44301404381474553856
Det: -8584014963274186752
Det: -1006901546341069440
Det: 44418210526843543552
Det: 93830086093328285696
Det: -14245348973205555200
Det: 4843409567414718464
Det: -2285110609815484416
Det: -14011414833711300608
Det: -54628011239628259328
Det: -23717301766440456192
Det: -11175100528110612480
Det: -30279828391545274368
Det: -61337611879214497792
Det: -49247373639396909056
Det: -33680928526853259264
Det: -9603028055247665152
Det: 54991655169981898752
Det: 286821155178941513728
Det: -157884767830545694720
Det: 3936698167662105088
Det: -190877134172570943488
Det: -4622161859788749824
Det: 35068676678947823616
Det: 97463904768362184704
Det: 15278033324834263040
D

ValueError: Maximum number of attempts reached. Unable to generate a suitable key matrix.